# AIO Q3 -- Fast and Approximate Matrix Multiplication  

## Overview

This question explores ways to speed up matrix multiplication using:

1. **The classical algorithm**  
2. **Sparsity** — exploiting zeros  
3. **Randomized approximation** — sampling a few informative rank-1 outer products  

We'll implement each method and **measure actual running times** to see when each approach wins.


In [ ]:
# Import necessary libraries
import time
import random
import math

## 1. Warm-Up: Classical Matrix Multiplication (Python) [15 points]

Write a Python function that computes the matrix product using the **standard triple nested loop**.

**Note:** Your implementation should use Python **lists** (list-of-lists representation), not NumPy arrays.


In [ ]:
def dense_multiply(A, B):
    """
    A: m×n list-of-lists
    B: n×p list-of-lists
    return: C = A @ B as an m×p list-of-lists
    """
    # YOUR CODE HERE [10 points]
    pass


**Question [5 points]:** State the number of scalar **multiplications** and **additions** it performs in terms of $m$, $n$, and $p$.
</br></br>
**Answer:**

## 2. Timing the Dense Algorithm

Use Python's `time` module to measure how long your dense multiplication takes on a $100 \times 100$ matrix multiplied by a $100 \times 100$ matrix.

**Run this code and record the time.**


In [ ]:
# Generate random test matrices
m, n, p = 100, 100, 100
A = [[random.random() for _ in range(n)] for _ in range(m)]
B = [[random.random() for _ in range(p)] for _ in range(n)]

# Time the dense algorithm
start = time.time()
C_dense = dense_multiply(A, B)
end = time.time()
print(f"Dense algorithm took {end - start:.4f} seconds")


## 3. Why Sparsity Helps [5 points]

Let $s_A$ and $s_B$ be the numbers of nonzero entries in $A$ and $B$.

**Question:** Describe (in words) what a "sparse-aware" algorithm should do differently from the dense algorithm to avoid wasted work on zeros.


**Answer:**

## 4. Sparse Multiplication by Iterating Over Nonzeros

Assume $A$ and $B$ are in sparse formats:

- `A_nonzeros = [(i, k, value), ...]`  
- `B_row_nonzeros = { k : [(j, value), ...] }`  

where `A_nonzeros` lists all nonzero entries of $A$, and  
`B_row_nonzeros[k]` lists all nonzeros in **row** $k$ of $B$.

### 4.1 Sparse Multiplication (Python) [15 points]

Write a Python function that computes $C = AB$ using only the nonzero entries.

**Hint:** For each nonzero entry $A[i,k]$, you need to update row $i$ of $C$ by adding $A[i,k] \cdot B[k,j]$ for each nonzero $B[k,j]$.


In [ ]:
def sparse_multiply(A_nonzeros, B_row_nonzeros, m, p):
    """
    A_nonzeros: list of (i, k, A[i][k])
    B_row_nonzeros: dict mapping k -> list of (j, B[k][j])
    m, p: dimensions of output matrix C (m×p)
    return: C as a dense list-of-lists
    """
    # YOUR CODE HERE
    pass


### 4.2 Helper Function (Provided)

Use this helper function to convert dense matrices to sparse format:


In [ ]:
def to_sparse(A):
    """
    A: m×n list-of-lists
    return: (A_nonzeros, A_row_nonzeros) where
      - A_nonzeros: list of (i, k, A[i][k]) for nonzero entries
      - A_row_nonzeros: dict mapping row k -> list of (j, A[k][j]) for nonzeros in that row
    """
    A_nonzeros = []
    A_row_nonzeros = {}
    
    for i in range(len(A)):
        for k in range(len(A[0])):
            if A[i][k] != 0:
                A_nonzeros.append((i, k, A[i][k]))
                if i not in A_row_nonzeros:
                    A_row_nonzeros[i] = []
                A_row_nonzeros[i].append((k, A[i][k]))
    
    return A_nonzeros, A_row_nonzeros


### 4.3 Timing Dense vs Sparse

Create two sparse matrices and compare the running times.

**Run this code and record both times and the speedup.**


In [ ]:
# Create sparse matrices (5% density)
def create_sparse_matrix(m, n, density=0.05):
    A = [[0.0 for _ in range(n)] for _ in range(m)]
    for i in range(m):
        for j in range(n):
            if random.random() < density:
                A[i][j] = random.random()
    return A

m, n, p = 200, 200, 200
A_sparse = create_sparse_matrix(m, n, density=0.05)
B_sparse = create_sparse_matrix(n, p, density=0.05)

# Time dense algorithm
start = time.time()
C_dense = dense_multiply(A_sparse, B_sparse)
end = time.time()
dense_time = end - start

# Convert to sparse format and time sparse algorithm
A_nonzeros, _ = to_sparse(A_sparse)
_, B_row_nonzeros = to_sparse(B_sparse)

start = time.time()
C_sparse = sparse_multiply(A_nonzeros, B_row_nonzeros, m, p)
end = time.time()
sparse_time = end - start

print(f"Dense algorithm: {dense_time:.4f} seconds")
print(f"Sparse algorithm: {sparse_time:.4f} seconds")
print(f"Speedup: {dense_time / sparse_time:.2f}x")


### 4.4 Runtime Analysis [15 points]

Let the nonzero entries of $A$ be grouped by column.  
For each column index $k$, define:

- $c_k =$ number of nonzeros in column $k$ of $A$  
- $r_k =$ number of nonzeros in row $k$ of $B$

**(a) [7 points]** Explain why the total number of arithmetic updates performed by the sparse algorithm is:

$$
\text{updates} \;=\; c_1 r_1 + c_2 r_2 + \cdots + c_n r_n.
$$
**Answer:**
</br></br>
**(b) [8 points]** Give a simple upper bound on this quantity using the facts:

$$
c_1 + \cdots + c_n = s_A,\qquad
r_1 + \cdots + r_n = s_B.
$$
**Answer:**
</br></br>
You do **not** need the tightest bound.  
Any reasonable bound in terms of $s_A$, $s_B$, and $n$ earns full credit.


## 5. When Is Sparse Faster Than Dense? [5 points]

The dense algorithm costs $\Theta(mnp)$ operations.  
The sparse algorithm does $\sum_k c_k r_k$ updates.

**Question:** Based on your timing experiments and the runtime analysis, give simple conditions on $s_A$, $s_B$, and the dimensions under which the sparse algorithm is **faster** than the dense algorithm in practice.
</br></br>
**Answer:**


## 6. Matrix Multiplication as a Sum of Outer Products [10 points]

Recall that matrix multiplication can be written as:

$$
AB = \sum_{i=1}^n A_{\bullet i} \otimes B_{i \bullet},
$$

where $A_{\bullet i}$ is column $i$ of $A$ and $B_{i\bullet}$ is row $i$ of $B$, and $\otimes$ denotes the outer product.

Suppose we pick index $i$ randomly with probability $p_i$, and define:

$$
X = \frac{1}{p_i} A_{\bullet i} \otimes B_{i\bullet}.
$$

**Show that** $\mathbb{E}[X] = AB$.

**Then explain:** Why might sampling a few such outer products (instead of all $n$) give a good approximation while being faster?
</br></br>
**Answer:**

## 7. The $t$-Sample Estimator

Define:

$$
\widehat{C} = \frac{1}{t}\sum_{j=1}^t X^{(j)},
$$

where each $X^{(j)}$ is an independent copy of $X$ from the previous question.

By linearity of expectation, $\widehat{C}$ is also an unbiased estimator of $AB$.

### 7.1 Python Implementation [15 points]

Write a function that computes the estimator above by drawing index $i$ with probability `p[i]` each time.

**Hint:** Use `random.choices(range(n), weights=p, k=1)[0]` to sample an index.


In [ ]:
def randomized_estimator(A, B, p, t):
    """
    A: m×n list-of-lists
    B: n×p list-of-lists
    p: list of sampling probabilities p[i] for i=0,...,n-1
    t: number of samples
    return: t-sample estimator of AB
    """
    # YOUR CODE HERE
    pass


### 7.2 Timing the Randomized Algorithm

Test your randomized estimator with uniform probabilities $p_i = 1/n$.

**Run this code and record the times.**


In [ ]:
m, n, p_dim = 100, 100, 100
A = [[random.random() for _ in range(n)] for _ in range(m)]
B = [[random.random() for _ in range(p_dim)] for _ in range(n)]

# Uniform probabilities
p = [1.0/n for _ in range(n)]

# Time for different values of t
for t in [5, 10, 20, 50]:
    start = time.time()
    C_approx = randomized_estimator(A, B, p, t)
    end = time.time()
    print(f"t={t}: {end - start:.4f} seconds")


### 7.3 Runtime [5 points]

**Question:** What is the runtime of your implementation in terms of $t$, $m$, and $p$ (the number of columns of $B$)?
</br></br>
**Answer:**


## 8. Measuring Approximation Quality

The **Frobenius norm** of a matrix $M$ is defined as:

$$
\|M\|_F = \sqrt{\sum_{i,j} M_{i,j}^2}
$$

It measures the "size" of a matrix by taking the square root of the sum of squared entries.

### 8.1 Computing Error [10 points]

Write a function to compute the Frobenius norm error:


In [ ]:
def frobenius_error(C_true, C_approx):
    """
    C_true: m×p list-of-lists (exact result)
    C_approx: m×p list-of-lists (approximation)
    return: ||C_true - C_approx||_F
    """
    # YOUR CODE HERE
    pass


### 8.2 Error vs Number of Samples [5 points]

Test how the error decreases as $t$ increases.

**Run this code.** What trend do you observe in the error as $t$ increases? Why does this make sense?


In [ ]:
m, n, p_dim = 50, 50, 50
A = [[random.random() for _ in range(n)] for _ in range(m)]
B = [[random.random() for _ in range(p_dim)] for _ in range(n)]

# Compute exact result
C_true = dense_multiply(A, B)

# Uniform probabilities
p = [1.0/n for _ in range(n)]

# Test different values of t
for t in [1, 2, 5, 10, 20, 50, 100]:
    C_approx = randomized_estimator(A, B, p, t)
    error = frobenius_error(C_true, C_approx)
    print(f"t={t}: error = {error:.4f}")
